# Getting Started: Keras with a Tensorflow Backend (In Anaconda)

Install Miniconda from https://conda.io/miniconda.html (Python 3.6) and launch the Anaconda prompt. Create a new environment and call it whatever you want (1), then activate this environment (2). Install spyder (3), then install Tensorflow using conda install, or failing that, pip (4). From the taskbar, click on Anaconda folder and then open spyder(your_env_name).

In [ ]:
'''(1) conda create -n your_env_name python=3.6'''

'''(2) activate your_env_name'''

'''(3) conda install spyder'''

'''(4) conda install -c conda-forge tensorflow /
       pip install tensorlow '''
    

Import relevant modules such as tensorflow, NumPy, SciPy and matplotlib. For image processing you may also need to install and import PIL.

In [ ]:
import scipy.io as sio

import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt


# MNIST

The MNIST data set is a great place to start. It contains labelled 28X28 pixel images of handwritten digits from 0 to 9. The machine trains on a large subset of the labeled data (Usually about 80%), then is tested on the remaining images without their labels. It's accuracy is measured by the proportion of digits from the test set which are correctly identified (computer prediction == label). Below is a heavily commented example of a dense network with two hidden layers which labels the test set with around 94% accuracy.


# Preparing the data:

Below we load some data (5000 out of 60000 entries) which is saved on the computer. Note the shape of the data as you will have to write its dimensions in your input layer. It is very common when building a model to have dimension errors. Sometimes data will need to be transposed or reshaped in order for the matrix multiplication going on in the hidden layers to work. (mxn)*(nxm) = (mxm): The number of columns in the first matrix must equal the number of rows in the second.
We then split the data into training and tests sets (70% training, 30% test).

In [ ]:
import scipy.io as sio

import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt


#Function to Load data ------------------------------------------------------

def loadMATData(file1):
    
    return sio.loadmat(file1)


#Load a small amoutn of MNIST Data-------------------------------------------------------------------

data = loadMATData('ex3data1.mat')   #The data is a (5000 X 2) matrix with column headers X,Y

features = data['X']      #Features are all the images of handwritten digits

labels = data['y']      #Labels describe above feaures

filter = labels ==10
labels[filter] = 0


#shuffle data---------------------------------------------------------------

ran = np.arange(features.shape[0])

np.random.shuffle(ran)

features = features[ran]

labels = labels[ran]


#Split data into training and test subsets------------------------------------

training_features = features[:3500]

training_labels = labels[:3500]

test_features = features[3501:]

test_labels = labels[3501:]

# Building a Sequential Model

Keras has two main model types, sequential and API. We will use a sequential model which you can easily create by using the .add() function or the keras.layers.dense() function to build up the model layer by layer.
The Keras functional API is designed for more complex models with multiple outputs, directed acyclic graphs or shared layers. It requires you to make input and output tensors which define the model.

In [ ]:
#Build model------------------------------------------------------------------

model = keras.Sequential([

        keras.layers.Dense(400, activation=tf.nn.relu),

        keras.layers.Dense(25, activation=tf.nn.relu),

        keras.layers.Dense(10, activation=tf.nn.softmax)
])

Above, we have two densely connected hidden layers of width 400 and 25, both with a relu activation function. Relu means rectified linear unit. The relu function lets all positive values pass through unchanged and sets any negative value to 0. Following that there is a softmax layer of width 10. This categorizes the digit into one of the  10 possible output categories that the images can be (categories are digits 0-9 and the correct digit will have output 1, all others output 0).

In [ ]:
#Compile model----------------------------------------------------------------

model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

The arguments of model.complile() above were chosen for the following reasons:

The Adam optimizer(alpha, beta1, beta2, epsilon) is a variation of the stochastic gradient descent algorithm which optimizes stochastic objective functions based on adaptive estimates of lower-order moments. Put simply, the learning rate is adjusted throughout. Instead of adapting the parameter learning rates based on the average first moment (the mean) as in RMSProp, Adam also makes use of the average of the second moments of the gradients (the uncentered variance).
It calculates an exponential moving average of the gradient and the squared gradient, and the parameters beta1 and beta2 control the decay rates of these moving averages. It can be used for non-stationary objectives and problems with very noisy and/or sparse gradients. Adam is presented in this paper: https://arxiv.org/abs/1412.6980v8

The loss function is equivalent to the cost function (J). It measures how well the machine learning model fits your data. It is the error (difference between estimated and true values) for an instance of data when fitting a gradient. When solving an optimization problem we are trying to minimise the loss function. If your outputs are ones and zeros you can use categorical_crossentropy, but if they are integers 1,2,3,4... is is better to use sparse_categorical_crossentropy.

In [ ]:
#Train the model--------------------------------------------------------------------

model.fit(training_features , training_labels, epochs=15)

The .fit() function trains the model on the data and labels from the training set. The number of epochs is the number of iterations. The loss should decrease with every epoch until it levels out. You can set a condition so that when the loss stops changing by significfant amounts, the algorithm stops.

In [ ]:
#Predict the test set digits--------------------------------------------------

predictions = model.predict(test_features)

The .predict() function 

In [ ]:
#Evaulate the accuracy of the model--------------------------------------------

count = 0

for i in range(0, len(test_labels)):
    
    pred = (np.argmax(predictions[i]))
    
    if test_labels[i][0] == pred:
        
        count +=1

print("Correct predictions: ", count)